In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import pickle
import numpy as np

In [3]:
from openslide import OpenSlide

In [4]:
from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi
from exact_sync.v1.api.products_api import ProductsApi

from exact_sync.v1.models import ImageSet, Product, AnnotationType, Image, Annotation
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

In [5]:
import sys
sys.path.append("../..")
from registration_tree import Rect, QuadTree

## Find and load slides from HDD

In [6]:
slide_list = {path.name: path for path in Path(r"D:\Datasets\ScannerStudy").glob("*/*/*.*") 
                      if path.suffix in [".svs", ".ndpi", ".tif"]}

slide_list

{'A_CCMCT_183715A_1.svs': WindowsPath('D:/Datasets/ScannerStudy/Aperio/CCMCT/A_CCMCT_183715A_1.svs'),
 'A_CCMCT_22108_1.svs': WindowsPath('D:/Datasets/ScannerStudy/Aperio/CCMCT/A_CCMCT_22108_1.svs'),
 'A_CCMCT_29609B_1.svs': WindowsPath('D:/Datasets/ScannerStudy/Aperio/CCMCT/A_CCMCT_29609B_1.svs'),
 'A_CCMCT_380609B_1.svs': WindowsPath('D:/Datasets/ScannerStudy/Aperio/CCMCT/A_CCMCT_380609B_1.svs'),
 'A_CCMCT_518711B_1.svs': WindowsPath('D:/Datasets/ScannerStudy/Aperio/CCMCT/A_CCMCT_518711B_1.svs'),
 'A_BB_563476_1.svs': WindowsPath('D:/Datasets/ScannerStudy/Aperio/Cyto/A_BB_563476_1.svs'),
 'A_BB_563479_1.svs': WindowsPath('D:/Datasets/ScannerStudy/Aperio/Cyto/A_BB_563479_1.svs'),
 'A_BB_568320_1.svs': WindowsPath('D:/Datasets/ScannerStudy/Aperio/Cyto/A_BB_568320_1.svs'),
 'A_BB_568381_1.svs': WindowsPath('D:/Datasets/ScannerStudy/Aperio/Cyto/A_BB_568381_1.svs'),
 'A_BB_574162_1.svs': WindowsPath('D:/Datasets/ScannerStudy/Aperio/Cyto/A_BB_574162_1.svs'),
 'Z_CCMCT_183715A_1.tif': Windo

## Connect to EXACT

In [7]:
configuration = Configuration()
configuration.username = 'marzahl' #'exact'
configuration.password = '###' #'exact'
configuration.host = "https://exact.cs.fau.de" #"http://127.0.0.1:8000"

client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)
product_api = ProductsApi(client)

## Download RefPoints annotation

In [8]:
annotation_types_ref_rect = None

for product in product_api.list_products(name="References").results:
    for annotation_type in annotation_types_api.list_annotation_types(product=product.id, name="BoundingBox").results:
        annotation_types_ref_rect = annotation_type
        
annotation_types_ref_rect

{'area_hit_test': False,
 'closed': True,
 'color_code': '#FF0000',
 'default_height': 5000,
 'default_width': 5000,
 'enable_blurred': False,
 'enable_concealed': False,
 'id': 412,
 'name': 'BoundingBox',
 'node_count': 0,
 'product': 105,
 'sort_order': 0,
 'vector_type': 1}

In [9]:
annotations = []

image_set = image_sets_api.list_image_sets(name="Cyto_Aperio", expand="product_set,product_set.annotationtype_set").results[0]

for image_id in image_set.images:
    
    image = images_api.retrieve_image(id=image_id)
    
    for anno in annotations_api.list_annotations(image=image_id,
                                                 deleted=False, fields="annotation_type,id,image,vector,unique_identifier").results:
        
        if anno.annotation_type != annotation_types_ref_rect.id:
            annotations.append([image.id, image.name, Path(image.name).stem.split("_")[2].split("_")[0], anno.id, anno.vector, anno.unique_identifier, anno.annotation_type])
        
annotations = pd.DataFrame(annotations, columns=["image_id", "image_name", "image_name_short", "id", "vector", "unique_identifier", "annotation_type"])
annotations

,image_id,image_name,image_name_short,id,vector,unique_identifier,annotation_type
0,10392,A_BB_574162_1.svs,574162,2754781,"{'x1': 5556, 'x2': 5656, 'y1': 5439, 'y2': 5539}",e4d98794-d4fb-404b-a4ae-4b312b468e8e,384
1,10392,A_BB_574162_1.svs,574162,2754782,"{'x1': 4671, 'x2': 4759, 'y1': 10586, 'y2': 10...",72f12897-7d3c-4458-a7d8-7fdc8e7176ab,385
2,10392,A_BB_574162_1.svs,574162,2754783,"{'x1': 5100, 'x2': 5208, 'y1': 15401, 'y2': 15...",a9d37e23-8267-49b1-a637-6c889bd79afa,386
3,10392,A_BB_574162_1.svs,574162,2754784,"{'x1': 5230, 'x2': 5336, 'y1': 21108, 'y2': 21...",42096c63-212d-4a51-bf00-e8d34de09f47,387
4,10392,A_BB_574162_1.svs,574162,2754785,"{'x1': 5387, 'x2': 5497, 'y1': 25652, 'y2': 25...",56efe99e-f891-4ad4-a9ee-e096ac3916aa,388
...,...,...,...,...,...,...,...
120,10378,A_BB_563476_1.svs,563476,2754901,"{'x1': 26701, 'x2': 26801, 'y1': 5443, 'y2': 5...",df3b63d5-da21-4fca-9336-cba6ef151289,407
121,10378,A_BB_563476_1.svs,563476,2754902,"{'x1': 27491, 'x2': 27612, 'y1': 11331, 'y2': ...",39f211cd-034b-4013-9271-439fef05afb9,408
122,10378,A_BB_563476_1.svs,563476,2754903,"{'x1': 26509, 'x2': 26624, 'y1': 16374, 'y2': ...",142f2230-ec26-4609-a5a5-93387387453d,409
123,10378,A_BB_563476_1.svs,563476,2754904,"{'x1': 26949, 'x2': 27048, 'y1': 22715, 'y2': ...",1e9cac70-7c73-4abb-8f8c-4170a0b1b35e,410


## Add some helper variables

In [10]:
annotations["x1"] = [vector['x1'] if type(vector) is dict else 0 for vector in annotations["vector"]]
annotations["y1"] = [vector['y1'] if type(vector) is dict else 0 for vector in annotations["vector"]]

annotations["x2"] = [vector['x2'] if type(vector) is dict else 0 for vector in annotations["vector"]]
annotations["y2"] = [vector['y2'] if type(vector) is dict else 0 for vector in annotations["vector"]]

annotations["center_x"] = [x1 + ((x2-x1) / 2) for x1, x2 in zip(annotations["x1"], annotations["x2"])]
annotations["center_y"] = [y1 + ((y2-y1) / 2) for y1, y2 in zip(annotations["y1"], annotations["y2"])]

annotations["anno_width"] = [x2-x1 for x1, x2 in zip(annotations["x1"], annotations["x2"])]
annotations["anno_height"]= [y2-y1 for y1, y2 in zip(annotations["y1"], annotations["y2"])]

## Load QuadTree for each scanner and transform coordinates

In [13]:
transformed_annotations_to_upload = []

for image_set_name in tqdm(["Cyto_2.0HT", "Cyto_Axio", "Cyto_S210"]):
    image_set = image_sets_api.list_image_sets(name=image_set_name, expand="product_set,product_set.annotationtype_set").results[0]

    for image_id in image_set.images:
        
        target_image = images_api.retrieve_image(id=image_id)
        
        source_annos = annotations[annotations["image_name_short"] == Path(target_image.name).stem.split("_")[2].split("_")[0]]
        
        source_name = list(source_annos["image_name"].unique())[0]
        target_name = target_image.name
        
        qtree = pickle.load(open(f'../../Cyto/{Path(source_name).stem}-To-{Path(target_name).stem}.pickle', "rb" ))
        qtree.source_slide = OpenSlide(str(slide_list[source_name]))
        qtree.target_slide = OpenSlide(str(slide_list[target_name]))
        
        for id, source_anno in source_annos.iterrows():

            box = [source_anno.center_x, source_anno.center_y, source_anno.anno_width, source_anno.anno_height]

            trans_box = qtree.transform_boxes(np.array([box]))[0]

            new_x1 = int(trans_box[0] - trans_box[2] // 2)
            new_y1 = int(trans_box[1] - trans_box[3] // 2)
            new_x2 = int(trans_box[0] + trans_box[2] // 2)
            new_y2 = int(trans_box[1] + trans_box[3] // 2)

            vector = {'x1': new_x1, 'x2': new_x2, 'y1': new_y1, 'y2': new_y2}

            anno = Annotation(annotation_type=source_anno.annotation_type, 
                                  vector=vector, 
                                  image=target_image.id,
                                 unique_identifier=source_anno.unique_identifier)

            transformed_annotations_to_upload.append(anno)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:07<00:00,  2.40s/it]


In [14]:
transformed_annotations_to_upload[:3]

[{'annotation_type': 384,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'image': 10383,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': 'e4d98794-d4fb-404b-a4ae-4b312b468e8e',
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 7273, 'x2': 7383, 'y1': 5858, 'y2': 5968},
  'verified_by_user': None},
 {'annotation_type': 385,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'image': 10383,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': '72f12897-7d3c-4458-a7d8-7fdc8e7176ab',
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 6301, 'x2': 6399, 'y1': 11591, 'y2': 11687},
  'verified_by_user': None},
 {'annotation_type': 386,
  'annotationversion_set': [],
  'blurred': None,
  'conce

## Check that no error occurred and afterwards upload annotations

In [15]:
annotations_api.create_annotation(body=transformed_annotations_to_upload)

[{'annotation_type': 384,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 2754906,
  'image': 10383,
  'last_edit_time': datetime.datetime(2021, 1, 10, 12, 28, 41, 527563),
  'last_editor': 1,
  'meta_data': None,
  'time': datetime.datetime(2021, 1, 10, 12, 28, 41, 527533),
  'unique_identifier': 'e4d98794-d4fb-404b-a4ae-4b312b468e8e',
  'uploaded_media_files': [],
  'user': 1,
  'vector': {'x1': 7273, 'x2': 7383, 'y1': 5858, 'y2': 5968},
  'verified_by_user': 'False'},
 {'annotation_type': 385,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 2754907,
  'image': 10383,
  'last_edit_time': datetime.datetime(2021, 1, 10, 12, 28, 41, 527640),
  'last_editor': 1,
  'meta_data': None,
  'time': datetime.datetime(2021, 1, 10, 12, 28, 41, 527627),
  'unique_identifier': '72f12897-7d3c-4458-a7d8-7fdc8e7176ab',
  'uploaded_media_files': [],